In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.impute import SimpleImputer
from catboost import CatBoostClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from pycaret.classification import *

In [ ]:
!pip install delayed

In [ ]:
!pip install pycaret

In [ ]:
!pip install catboost

## Loadiang data

In [ ]:
!wget https://github.com/dsbattle/hackathon-mkb/raw/master/train_dataset_hackathon_mkb.csv
!wget https://github.com/dsbattle/hackathon-mkb/raw/master/test_dataset_hackathon_mkb.csv

In [ ]:
train = pd.read_csv('train_dataset_hackathon_mkb.csv', sep=';', encoding='cp1251')
test = pd.read_csv('test_dataset_hackathon_mkb.csv', sep=';', encoding='cp1251')

In [ ]:
clf = setup(train, target = 'TARGET')

In [ ]:
train.head()

## Delete bad features

In [ ]:
train = train.drop(columns='BIRTHDATE', axis = 1)
train = train.drop(columns='SIGN_DATE' , axis = 1)
train = train.drop(columns='id_client',axis = 1)
train = train.drop_duplicates()

In [ ]:
#train.isnull().sum()

In [ ]:
#sns.countplot(train['TARGET'])

## preproccesing features

In [ ]:
le = LabelEncoder()
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
categorical = ['object']
for cols in list((train.select_dtypes(include=numerics)).columns.values):
    imp = SimpleImputer(missing_values=np.nan, strategy='mean')
    imp.fit(train[cols].values.reshape(-1,1))
    train[cols] = imp.transform(train[cols].values.reshape(-1,1))
    train[cols] = train[cols].astype(int)
    
for cols in list((train.select_dtypes(include=categorical)).columns.values):
    imp = SimpleImputer(strategy="most_frequent")
    imp.fit(train[cols].values.reshape(-1,1))
    train[cols] = imp.transform(train[cols].values.reshape(-1,1))
for cols in list((train.select_dtypes(include=categorical)).columns.values):
    train[cols] =  le.fit_transform(train[cols])  

In [ ]:
k_means = train.loc[:, ["OKVED_CODE", "OKTMO_FED", "CITIZENSHIP_NAME"]]
kmeans = KMeans(n_clusters=6)
train["Cluster"] = kmeans.fit_predict(k_means)
train["Cluster"] = train["Cluster"].astype(int)
k_means.head()

,OKVED_CODE,OKTMO_FED,CITIZENSHIP_NAME
0,212,40,2
1,374,34,2
2,229,33,2
3,212,34,2
4,374,34,2


In [ ]:
train

,id_contract,IP_flag,TARGET,F1100,F1110,F1150,F1160,F1170,F1180,F1190,F1200,F1210,F1220,F1230,F1240,F1250,F1260,F1300,F1310,F1320,F1350,F1360,F1370,F1400,F1410,F1420,F1450,F1500,F1510,F1520,F1530,F1550,F1600,F1700,F2100,F2110,F2120,F2200,F2210,F2220,...,MANAGERINNCOUNT,OKVED_CODE,PLAINTIFF_CASESNUMBER_YEAR,PLAINTIFF_SUM_YEAR,DEFENDANT_CASESNUMBER_YEAR,DEFENDANT_SUM_YEAR,THIRDOROTHERPERSON_YEAR,PLAINTIFF_CASESNUMBER_EVER,PLAINTIFF_SUM_EVER,DEFENDANT_CASESNUMBER_EVER,DEFENDANT_SUM_EVER,THIRDOROTHERPERSON_EVER,ADMITTEDNUMBER_233_YEAR,NOTADMITTEDNUMBER_233_YEAR,WINNERNUMBER_233_YEAR,SIGNEDNUMBER_233_YEAR,SUM_233_YEAR,ADMITTEDNUMBER_233_EVER,NOTADMITTEDNUMBER_233_EVER,WINNERNUMBER_233_EVER,SIGNEDNUMBER_233_EVER,SUM_233_EVER,ADMITTEDNUMBER_95_YEAR,NOTADMITTEDNUMBER_95_YEAR,WINNERNUMBER_95_YEAR,SIGNEDNUMBER_95_YEAR,SUM_95_YEAR,ADMITTEDNUMBER_95_EVER,NOTADMITTEDNUMBER_EVER,WINNERNUMBER_95_EVER,SIGNEDNUMBER_95_EVER,SUM_95_EVER,FLAG_DISQUALIFICATION,COUNT_CHANGE_YEAR,COUNT_CHANGE_EVER,AGE,SEX_NAME,CITIZENSHIP_NAME,Cluster,mean
0,1,0,0,1298961000,2154000,1125573000,49817797,150010000,134109954,21224000,10482866000,105901000,127510000,9477348000,440671000,331298000,138000,1960896000,10007000,-46634000,599497000,500000,1028878000,340135000,2537964726,340135000,644768920,9480796000,2499973000,6907960000,206810166,115750392,11781827000,11781827000,2370882000,52941040000,50570158000,631752000,1083075000,656055000,...,0,212,5,11547233,1,2727485,0,11,16716350,4,7649294,1,48,7,22,17,52939799,124,17,51,37,229193594,255,4,142,127,111204882,651,16,354,323,254479048,1,1,2,41,1,2,2,1.734895
1,2,1,0,3184581862,113635671,2287078366,49817797,3389027122,134109954,720544875,970536232,167589952,30301028,637034391,364827217,111747382,27081752,1244925898,817355846,-27283793,3076378014,61200815,450373225,1813306899,2537964726,316611617,644768920,964268144,558756209,649607853,206810166,115750392,2829704219,2830592767,1350922338,3430807318,2653296332,191548133,1571201106,236289626,...,1,374,5,11547233,1,2727485,0,11,16716350,4,7649294,1,48,7,22,17,52939799,124,17,51,37,229193594,255,4,142,127,111204882,651,16,354,323,254479048,1,1,2,41,1,2,0,1.734895
2,3,0,0,73374000,113635671,73374000,49817797,3389027122,134109954,720544875,21610000,2378000,30301028,17642000,1000000,553000,37000,90736000,100000,-27283793,6780000,61200815,83856000,1813306899,2537964726,316611617,644768920,4248000,558756209,4171000,77000,115750392,94984000,94984000,9315000,52184000,42869000,2569000,1571201106,6746000,...,1,229,6,2762852,1,794,3,21,5408695,39,882647,4,1,0,1,1,1174998,2,0,2,2,2738217,38,0,38,39,4209963,169,0,169,168,18351739,1,1,1,41,1,2,2,1.734895
3,4,0,0,1937488000,122828000,610328000,49817797,809426000,356828000,38078000,5786965000,12001000,71000,1232486000,78603000,4459193000,4611000,2685905000,1516514000,-27283793,3076378014,65834000,1103557000,627253000,2537964726,689000,626564000,4411295000,393000,3092581000,206810166,115750392,7724453000,7724453000,16940541000,40284663000,23344122000,931118000,16009423000,236289626,...,1,212,5,11547233,1,2727485,0,11,16716350,4,7649294,1,48,7,22,17,52939799,124,17,51,37,229193594,255,4,142,127,111204882,651,16,354,323,254479048,1,1,2,41,1,2,2,1.734895
4,5,1,0,3184581862,113635671,2287078366,49817797,3389027122,134109954,720544875,970536232,167589952,30301028,637034391,364827217,111747382,27081752,1244925898,817355846,-27283793,3076378014,61200815,450373225,1813306899,2537964726,316611617,644768920,964268144,558756209,649607853,206810166,115750392,2829704219,2830592767,1350922338,3430807318,2653296332,191548133,1571201106,236289626,...,1,374,5,11547233,1,2727485,0,11,16716350,4,7649294,1,48,7,22,17,52939799,124,17,51,37,229193594,255,4,142,127,111204882,651,16,354,323,254479048,1,1,2,41,1,2,0,1.734895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [ ]:
scale_features = [col for col in train.columns if 'F'  in col]

In [ ]:
corr = data__not_approved.corr()
ax , fig  = plt.subplots(figsize =(10 , 8 ))
sns.heatmap(corr ,vmin = -1 , cmap='coolwarm', annot = True)
plt.show()

In [ ]:
corr[abs(corr['TARGET']) > 0.1]['TARGET']

TARGET                        1.000000
F1160                        -0.110808
F1180                        -0.109962
F1200                        -0.103927
F1450                        -0.165524
F1520                        -0.117995
F2110                        -0.128008
F2120                        -0.131946
F2210                        -0.122650
F1520_LAG1                   -0.111780
F2110_LAG1                   -0.129720
F2120_LAG1                   -0.133352
F2210_LAG1                   -0.129638
EGRPOINCLUDED                -0.192296
OKATO_REGIONCODE             -0.108216
OKTMO_CODE                   -0.109015
COUNTCOOWNERFCSM             -0.133818
COUNTCOOWNERROSSTAT          -0.128452
PLAINTIFF_SUM_YEAR            0.123562
PLAINTIFF_CASESNUMBER_EVER   -0.103085
THIRDOROTHERPERSON_EVER      -0.100457
NOTADMITTEDNUMBER_233_YEAR    0.112582
SUM_233_YEAR                  0.174278
ADMITTEDNUMBER_233_EVER       0.148576
NOTADMITTEDNUMBER_233_EVER    0.133636
SUM_233_EVER             

In [ ]:
corr = data_approved.corr()
ax , fig  = plt.subplots(figsize =(10 , 8 ))
sns.heatmap(corr ,vmin = -1 , cmap='coolwarm', annot = True)
plt.show()

In [ ]:
corr[abs(corr['TARGET']) > 0.1]['TARGET']

TARGET                        1.000000
OKATO_REGIONCODE             -0.105330
OKATO_FED                    -0.204547
OKTMO_CODE                   -0.103992
OKTMO_FED                    -0.205652
DEFENDANT_CASESNUMBER_YEAR    0.215990
THIRDOROTHERPERSON_YEAR      -0.158379
THIRDOROTHERPERSON_EVER      -0.168107
AGE                          -0.347063
Name: TARGET, dtype: float64

## Modeling 

In [ ]:
X = train.drop(labels=['TARGET'], axis=1)
y = train['TARGET']

In [ ]:
X_train , X_test , y_train , y_test = train_test_split(X,y, train_size = 0.8, random_state = 42)

In [ ]:
model = CatBoostClassifier(iterations=1500,verbose=False)
model.fit(X_train,y_train)

In [ ]:
#lgb = LGBMClassifier(n_estimators=500)
#lgb.fit(X_train,y_train)

In [ ]:
#xgb = XGBClassifier(n_estimators=2000)
#xgb.fit(X_train,y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
roc_auc_score(y_test , y_pred)

0.8906113006541666

In [ ]:
def plot_feature_importance(importance,names,model_type):
  feature_importance = np.array(importance)
  feature_names = np.array(names)

#Create a DataFrame using a Dictionary
  data={'feature_names':feature_names,'feature_importance':feature_importance}
  fi_df = pd.DataFrame(data)
  fi_df.sort_values(by=['feature_importance'], ascending=False,inplace=True)

#Define size of bar plot
  plt.figure(figsize=(20,12))
#Plot Searborn bar chart
  sns.barplot(x=fi_df['feature_importance'], y=fi_df['feature_names'])
#Add chart labels
  plt.title(model_type + 'FEATURE IMPORTANCE')
  plt.xlabel('FEATURE IMPORTANCE')
  plt.ylabel('FEATURE NAMES')

In [ ]:
plot_feature_importance(model.feature_importances_,X_train.columns,'CatBoost')

## Test preproccesing before prediction

In [ ]:
test = test.drop(columns='BIRTHDATE', axis = 1)
test = test.drop(columns='SIGN_DATE' , axis = 1)
test = test.drop(columns='id_client',axis = 1)
test = test.drop_duplicates()

for cols in list((test.select_dtypes(include=numerics)).columns.values):
    imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
    imp.fit(test[cols].values.reshape(-1,1))
    test[cols] = imp.transform(test[cols].values.reshape(-1,1))
    
for cols in list((test.select_dtypes(include=categorical)).columns.values):
    imp = SimpleImputer(strategy="most_frequent")
    imp.fit(test[cols].values.reshape(-1,1))
    test[cols] = imp.transform(test[cols].values.reshape(-1,1))
for cols in list((test.select_dtypes(include=categorical)).columns.values):
    test[cols] =  le.fit_transform(test[cols])

In [ ]:
k_means = test.loc[:, ["OKVED_CODE", "OKTMO_FED", "CITIZENSHIP_NAME"]]
kmeans = KMeans(n_clusters=6)
test["Cluster"] = kmeans.fit_predict(k_means)
test["Cluster"] = test["Cluster"].astype(int)
k_means.head()

,OKVED_CODE,OKTMO_FED,CITIZENSHIP_NAME
0,308,30.0,1
1,266,30.0,1
2,150,34.0,1
3,450,31.0,1
4,210,40.0,1


In [ ]:
test

In [ ]:
test['TARGET'] = model.predict_proba(test)[:, 1]

In [ ]:
test

,id_contract,IP_flag,F1100,F1110,F1150,F1160,F1170,F1180,F1190,F1200,F1210,F1220,F1230,F1240,F1250,F1260,F1300,F1310,F1320,F1350,F1360,F1370,F1400,F1410,F1420,F1450,F1500,F1510,F1520,F1530,F1550,F1600,F1700,F2100,F2110,F2120,F2200,F2210,F2220,F2300,...,MANAGERINNCOUNT,OKVED_CODE,PLAINTIFF_CASESNUMBER_YEAR,PLAINTIFF_SUM_YEAR,DEFENDANT_CASESNUMBER_YEAR,DEFENDANT_SUM_YEAR,THIRDOROTHERPERSON_YEAR,PLAINTIFF_CASESNUMBER_EVER,PLAINTIFF_SUM_EVER,DEFENDANT_CASESNUMBER_EVER,DEFENDANT_SUM_EVER,THIRDOROTHERPERSON_EVER,ADMITTEDNUMBER_233_YEAR,NOTADMITTEDNUMBER_233_YEAR,WINNERNUMBER_233_YEAR,SIGNEDNUMBER_233_YEAR,SUM_233_YEAR,ADMITTEDNUMBER_233_EVER,NOTADMITTEDNUMBER_233_EVER,WINNERNUMBER_233_EVER,SIGNEDNUMBER_233_EVER,SUM_233_EVER,ADMITTEDNUMBER_95_YEAR,NOTADMITTEDNUMBER_95_YEAR,WINNERNUMBER_95_YEAR,SIGNEDNUMBER_95_YEAR,SUM_95_YEAR,ADMITTEDNUMBER_95_EVER,NOTADMITTEDNUMBER_EVER,WINNERNUMBER_95_EVER,SIGNEDNUMBER_95_EVER,SUM_95_EVER,FLAG_DISQUALIFICATION,COUNT_CHANGE_YEAR,COUNT_CHANGE_EVER,AGE,SEX_NAME,CITIZENSHIP_NAME,Cluster,TARGET
0,17892,1,161000.0,214939000.0,43000.0,3592000.0,8.211189e+09,119000.0,214515000.0,1852000.0,70000.0,385000.0,1782000.0,1743000.0,6273000.0,71000.0,10000.0,10000.0,-295651000.0,370000.0,1000.0,434000.0,3290000.0,3290000.0,275441000.0,2.819541e+09,1408000.0,2330000.0,1408000.0,-524000.0,9012000.0,1852000.0,1852000.0,4767000.0,33153000.0,28386000.0,490000.0,4277000.0,4.036529e+09,338000.0,...,1.0,308,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,16.0,0.0,1.0,0.0,0.0,16.0,0.0,1.0,0.0,0.000000e+00,1.0,1.0,1.0,33.0,1,1,0,0.035404
1,17893,0,1906000.0,214939000.0,1906000.0,3592000.0,8.211189e+09,119000.0,214515000.0,14838000.0,4392000.0,385000.0,727000.0,1743000.0,9719000.0,71000.0,15222000.0,150000.0,-295651000.0,370000.0,1000.0,15072000.0,3290000.0,3290000.0,275441000.0,2.819541e+09,1522000.0,2330000.0,1522000.0,-524000.0,9012000.0,16744000.0,16744000.0,2998000.0,310151000.0,307153000.0,2998000.0,4277000.0,4.036529e+09,2998000.0,...,2.0,266,2.0,0.0,0.0,0.0,0.0,7.0,4726558.0,0.0,0.0,0.0,181.0,0.0,77.0,47.0,17883526.0,1182.0,14.0,589.0,533.0,169299901.0,1204.0,2.0,857.0,767.0,138194657.0,10240.0,69.0,7439.0,7100.0,1.187411e+09,1.0,1.0,1.0,33.0,1,1,0,0.006772
2,17894,0,147000.0,214939000.0,147000.0,3592000.0,8.211189e+09,119000.0,214515000.0,95529000.0,29812000.0,7000.0,56538000.0,504000.0,8430000.0,238000.0,4145000.0,2538000.0,-295651000.0,370000.0,1000.0,1607000.0,3290000.0,3290000.0,275441000.0,2.819541e+09,91531000.0,2330000.0,91531000.0,-524000.0,9012000.0,95676000.0,95676000.0,48975000.0,506023000.0,457048000.0,1753000.0,47222000.0,4.036529e+09,1771000.0,...,1.0,150,10.0,2272145.0,2.0,176875.0,0.0,103.0,27542049.0,14.0,1743130.0,2.0,31.0,2.0,14.0,11.0,3681170.0,177.0,24.0,73.0,63.0,46085424.0,36.0,10.0,7.0,7.0,2375210.0,764.0,50.0,362.0,344.0,9.836260e+07,1.0,1.0,3.0,33.0,1,1,2,0.022229
3,17895,0,161000.0,214939000.0,43000.0,3592000.0,8.211189e+09,119000.0,214515000.0,1906000.0,304000.0,385000.0,682000.0,1743000.0,920000.0,71000.0,1038000.0,10000.0,-295651000.0,370000.0,1000.0,434000.0,95000.0,95000.0,275441000.0,2.819541e+09,774000.0,2330000.0,774000.0,-524000.0,9012000.0,1907000.0,1907000.0,4767000.0,9793000.0,8731000.0,1062000.0,4277000.0,4.036529e+09,1027000.0,...,1.0,450,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,16.0,0.0,1.0,0.0,0.0,16.0,0.0,1.0,0.0,0.000000e+00,1.0,1.0,1.0,33.0,1,1,1,0.287411
4,17896,0,26000.0,214939000.0,43000.0,3592000.0,2.600000e+04,119000.0,214515000.0,2735000.0,1324000.0,385000.0,1410000.0,1743000.0,1000.0,71000.0,1220000.0,10000.0,-295651000.0,370000.0,1000.0,434000.0,3290000.0,3290000.0,275441000.0,2.819541e+09,1541000.0,179000.0,1362000.0,-524000.0,9012000.0,2761000.0,2761000.0,4767000.0,6407000.0,6301000.0,106000.0,4277000.0,4.036529e+09,90000.0,...,2.0,210,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,16.0,0.0,1.0,0.0,0.0,16.0,0.0,1.0,0.0,0.000000e+00,1.0,1.0,1.0,33.0,1,1,2,0.22

In [ ]:
test[['id_contract', 'TARGET']].to_csv('submit_file6.csv', sep=';', index=False)
test[['id_contract', 'TARGET']].head()